# Uncertainty calculation for model: WK2

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()

In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites, images and settings

In [4]:
#definet the color pallette
vibgYOR = ['330044', '220066', '1133cc', '33dd00', 'ffda21', 'ff6622', 'd10000']
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec");
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)
# generete the pixel area map
pixelArea = ee.Image.pixelArea().divide(10000) # to ha unit
# load the biome layer
biomeLayer = compositeImage.select("WWF_Biome")
biomeMask = biomeLayer.mask(biomeLayer.neq(98)).mask(biomeLayer.neq(99)).gt(0)
# load the mean maps for present and potential
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the biomass density layers
mergedAGB_PresentMean =  ee.Image("users/leonidmoore/ForestBiomass/WalkerMap/reprojected_Walker_map_1km").unmask() 
mergedAGB_PotentialMean = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK2_Potential_density_Ensambled_Mean").unmask()

# define the standardized projection
stdProj = mergedAGB_PresentMean.projection()
# load the two forest cover layer for existing and potential forest
presentForestCover = compositeImage.select('PresentTreeCover').unmask()# make sure it's in  0-1 scale
potentialForestCover = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask() # make sure it's in  0-1 scale

# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialForestCover.gt(0)

## 2 Calculate the present and potential AGB

In [5]:
# check the difference of the two density maps
potentialHigher = mergedAGB_PotentialMean.multiply(pixelArea).subtract(mergedAGB_PresentMean.multiply(pixelArea)).gte(0)
potentialLower = mergedAGB_PotentialMean.multiply(pixelArea).subtract(mergedAGB_PresentMean.multiply(pixelArea)).lt(0)
# replace the lower potential value by present biomass density value
potentialAGB_Density = mergedAGB_PresentMean.multiply(potentialLower).add(mergedAGB_PotentialMean.multiply(potentialHigher))
presentAGB_Density = mergedAGB_PresentMean
# get the abs of present and potential AGB
presentAGB_Abs = presentAGB_Density.multiply(pixelArea).multiply(presentMask).divide(1000000000)
potentialAGB_Abs = potentialAGB_Density.multiply(pixelArea).multiply(potentialMask).divide(1000000000)

# presentAGB_Abs_Sum = presentAGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                  geometry = unboundedGeo,
#                                                  crs = 'EPSG:4326',
#                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                  maxPixels = 1e9)
# # print the estimation out
# print(colored('The present AGB:', 'blue', attrs=['bold']),presentAGB_Abs_Sum.getInfo())
# potentialAGB_Abs_Sum = potentialAGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                      geometry = unboundedGeo,
#                                                      crs = 'EPSG:4326',
#                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                      maxPixels = 1e9)
# # print the estimation out
# print(colored('The potential AGB:', 'blue', attrs=['bold']),potentialAGB_Abs_Sum.getInfo())

## 3 Calculate the Upper and Lower of present and potential AGB

In [6]:
# load the present prediction lower and upper layer
# this is the interval index, in %. divide by 2 and 100 to make it into the upper and lower boundary
mergedPredictionInterval = ee.Image("users/leonidmoore/ForestBiomass/WalkerMap/reprojected_Walker_map_1km_Uncertainty").unmask().divide(200)
presentAGB_Lower = mergedAGB_PresentMean.subtract(mergedAGB_PresentMean.multiply(mergedPredictionInterval))
presentAGB_Upper = mergedAGB_PresentMean.add(mergedAGB_PresentMean.multiply(mergedPredictionInterval))

# get the upper and lower layer
mergedAGB_PotentialLower = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK2_Potential_density_Ensambled_Percentile").select(['lower']).unmask() 
mergedAGB_PotentialUpper = ee.Image("users/nordmannmoore/ForestBiomass/RemoteSensingModel/EnsambleMaps/Predicted_WK2_Potential_density_Ensambled_Percentile").select(['upper']).unmask() 

mergedAGB_PotentialLower1 = presentAGB_Lower.multiply(potentialLower).add(mergedAGB_PotentialLower.multiply(potentialHigher))
mergedAGB_PotentialUpper1 = presentAGB_Upper.multiply(potentialLower).add(mergedAGB_PotentialUpper.multiply(potentialHigher))

# define the masks to mask the present and potential lower maps
potentialAGB_Lower_Larger = mergedAGB_PotentialLower1.subtract(presentAGB_Lower).gte(0) # potential is larger than present mean
potentialAGB_Lower_Smaller = mergedAGB_PotentialLower1.subtract(presentAGB_Lower).lt(0)
# define the masks to mask the present and potential upper maps
potentialAGB_Upper_Larger = mergedAGB_PotentialUpper1.subtract(presentAGB_Upper).gte(0) # potential is larger than present upper
potentialAGB_Upper_Smaller = mergedAGB_PotentialUpper1.subtract(presentAGB_Upper).lt(0)

# replace the lower potential value by present biomass density value
potentialAGB_AdjLower = mergedAGB_PotentialLower.multiply(potentialAGB_Lower_Larger).add(presentAGB_Lower.multiply(potentialAGB_Lower_Smaller))
potentialAGB_AdjUpper = mergedAGB_PotentialUpper.multiply(potentialAGB_Upper_Larger).add(presentAGB_Upper.multiply(potentialAGB_Upper_Smaller))

# present lower and higher
presentAGB_Lower_Abs = presentAGB_Lower.multiply(pixelArea).multiply(presentMask).divide(1000000000)
presentAGB_Upper_Abs = presentAGB_Upper.multiply(pixelArea).multiply(presentMask).divide(1000000000)

# abs potential lower and higher
potentialAGB_Lower_Abs = potentialAGB_AdjLower.multiply(pixelArea).multiply(potentialMask).divide(1000000000)
potentialAGB_Upper_Abs = potentialAGB_AdjUpper.multiply(pixelArea).multiply(potentialMask).divide(1000000000)

In [7]:
# # Calculate the present AGB lower
# presentAGB_Lower_Sum = presentAGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                          geometry = unboundedGeo,
#                                                          crs = 'EPSG:4326',
#                                                          crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                          maxPixels = 1e9)

# # print the estimation out
# print(colored('The present AGB Lower:', 'blue', attrs=['bold']),presentAGB_Lower_Sum.getInfo())

# # Calculate the present AGB upper
# presentAGB_Upper_Sum = presentAGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                          geometry = unboundedGeo,
#                                                          crs = 'EPSG:4326',
#                                                          crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                          maxPixels = 1e9)

# # print the estimation out
# print(colored('The present AGB Upper:', 'blue', attrs=['bold']),presentAGB_Upper_Sum.getInfo())

# potentialAGB_Lower_Sum = potentialAGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                              geometry = unboundedGeo,
#                                                              crs = 'EPSG:4326',
#                                                              crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                              maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential AGB Lower:', 'blue', attrs=['bold']),potentialAGB_Lower_Sum.getInfo())

# potentialAGB_Upper_Sum = potentialAGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                              geometry = unboundedGeo,
#                                                              crs = 'EPSG:4326',
#                                                              crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                              maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential AGB Upper:', 'blue', attrs=['bold']),potentialAGB_Upper_Sum.getInfo())


## 4 Calculate the Upper and Lower of present and potential Root and TGB

In [8]:
# load the root shoot map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
rootShootRatioLower = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_percentile_Map").select('lower').unmask()
rootShootRatioUpper = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_percentile_Map").select('upper').unmask()
# 
presentRoot_Lower_Abs = presentAGB_Lower_Abs.multiply(rootShootRatioLower).mask(presentMask)
presentRoot_Upper_Abs = presentAGB_Upper_Abs.multiply(rootShootRatioUpper).mask(presentMask)

potentialRoot_Lower_Abs = potentialAGB_Lower_Abs.multiply(rootShootRatioLower).mask(potentialMask)
potentialRoot_Upper_Abs = potentialAGB_Upper_Abs.multiply(rootShootRatioUpper).mask(potentialMask)

presentRoot_Abs = presentAGB_Abs.multiply(rootShootRatio).mask(presentMask)
potentialRoot_Abs = potentialAGB_Abs.multiply(rootShootRatio).mask(potentialMask)

presentTGB_Abs = presentAGB_Abs.multiply(rootShootRatio).add(presentAGB_Abs)#.multiply(presentMask)
potentialTGB_Abs = potentialAGB_Abs.multiply(rootShootRatio).add(potentialAGB_Abs)#.multiply(potentialMask)

presentTGB  = presentAGB_Density.multiply(rootShootRatio.add(1))
# density 
presentRoot = presentAGB_Density.multiply(rootShootRatio)
presentRoot_Lower = presentAGB_Lower.multiply(rootShootRatioLower)
presentRoot_Upper = presentAGB_Upper.multiply(rootShootRatioLower)

potentialRoot_Lower = potentialAGB_AdjLower.multiply(rootShootRatioLower)
potentialRoot_Upper = potentialAGB_AdjUpper.multiply(rootShootRatioLower)

presentTGB_Lower = presentAGB_Lower.multiply(rootShootRatioLower).add(presentAGB_Lower)
presentTGB_Upper = presentAGB_Upper.multiply(rootShootRatioLower).add(presentAGB_Upper)

potentialTGB_Lower = potentialAGB_AdjLower.multiply(rootShootRatioLower).add(potentialAGB_AdjLower)
potentialTGB_Upper = potentialAGB_AdjUpper.multiply(rootShootRatioLower).add(potentialAGB_AdjUpper)

# presentTGB_Abs_Sum = presentTGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                  geometry = unboundedGeo,
#                                                  crs = 'EPSG:4326',
#                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                  maxPixels = 1e9)

# # print the estimation out
# print(colored('The present TGB:', 'blue', attrs=['bold']),presentTGB_Abs_Sum.getInfo())

# potentialTGB_Abs_Sum = potentialTGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                      geometry = unboundedGeo,
#                                                      crs = 'EPSG:4326',
#                                                      crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                      maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential TGB:', 'blue', attrs=['bold']),potentialTGB_Abs_Sum.getInfo())

# presentRoot_Abs_Sum = presentRoot_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                    geometry = unboundedGeo,
#                                                    crs = 'EPSG:4326',
#                                                    crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                    maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Roots:', 'blue', attrs=['bold']),presentRoot_Abs_Sum.getInfo())

# potentialRoot_Abs_Sum = potentialRoot_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Roots:', 'blue', attrs=['bold']),potentialRoot_Abs_Sum.getInfo())

# presentRoot_Lower_Sum = presentRoot_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Roots Lower:', 'blue', attrs=['bold']),presentRoot_Lower_Sum.getInfo())

# presentRoot_Upper_Sum = presentRoot_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Roots Upper:', 'blue', attrs=['bold']),presentRoot_Upper_Sum.getInfo())

# potentialRoot_Lower_Sum = potentialRoot_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Roots Lower:', 'blue', attrs=['bold']),potentialRoot_Lower_Sum.getInfo())
# potentialRoot_Upper_Sum = potentialRoot_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Roots Upper:', 'blue', attrs=['bold']),potentialRoot_Upper_Sum.getInfo())

In [9]:
presentTGB_Lower_Abs = presentAGB_Lower_Abs.multiply(rootShootRatioLower.add(1))
presentTGB_Upper_Abs = presentAGB_Upper_Abs.multiply(rootShootRatioUpper.add(1))

potentialTGB_Lower_Abs = potentialAGB_Lower_Abs.multiply(rootShootRatioLower.add(1))
potentialTGB_Upper_Abs = potentialAGB_Upper_Abs.multiply(rootShootRatioUpper.add(1))

# presentTGB_Lower_Sum = presentTGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present TGB Lower:', 'blue', attrs=['bold']),presentTGB_Lower_Sum.getInfo())

# presentTGB_Upper_Sum = presentTGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present TGB Upper:', 'blue', attrs=['bold']),presentTGB_Upper_Sum.getInfo())

# potentialTGB_Lower_Sum = potentialTGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential TGB Lower:', 'blue', attrs=['bold']),potentialTGB_Lower_Sum.getInfo())
# potentialTGB_Upper_Sum = potentialTGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential TGB Upper:', 'blue', attrs=['bold']),potentialTGB_Upper_Sum.getInfo())

## 5 Calculate the Upper and Lower of present and potential Root and PGB

In [10]:
# load the dead wood and litter layer
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()
deadWoodLitterRatioLower = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Lower_Map").unmask()
deadWoodLitterRatioUpper = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Upper_Map").unmask()
# calculate the present and potential PGB
presentPGB_Abs = presentTGB_Abs.multiply(deadWoodLitterRatio)
potentialPGB_Abs = potentialTGB_Abs.multiply(deadWoodLitterRatio)

# calculate the present and potential dead wood and litter
presentLitter_Abs = presentTGB_Abs.multiply(deadWoodLitterRatio.subtract(1))
potentialLitter_Abs = potentialTGB_Abs.multiply(deadWoodLitterRatio.subtract(1))

# calculate the present Dead wood and litter
presentLitter_Lower_Abs = presentTGB_Lower_Abs.multiply(deadWoodLitterRatioLower.subtract(1))
presentLitter_Upper_Abs = presentTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper.subtract(1))
# calculate the potential dead wood and litter
potentialLitter_Lower_Abs = potentialTGB_Lower_Abs.multiply(deadWoodLitterRatioLower.subtract(1))
potentialLitter_Upper_Abs = potentialTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper.subtract(1))
# get the densities
presentPGB_Lower = presentTGB_Lower.multiply(deadWoodLitterRatioLower)
presentPGB_Upper = presentTGB_Upper.multiply(deadWoodLitterRatioUpper)

potentialPGB_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioLower)
potentialPGB_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioUpper)

presentLitter_Lower = presentTGB_Lower.multiply(deadWoodLitterRatioLower.subtract(1))
presentLitter_Upper = presentTGB_Upper.multiply(deadWoodLitterRatioUpper.subtract(1))

potentialLitter_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioLower.subtract(1))
potentialLitter_Lower = potentialTGB_Lower.multiply(deadWoodLitterRatioUpper.subtract(1))

# presentPGB_Abs_Sum = presentPGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present PGB:', 'blue', attrs=['bold']),presentPGB_Abs_Sum.getInfo())

# potentialPGB_Abs_Sum = potentialPGB_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential PGB:', 'blue', attrs=['bold']),potentialPGB_Abs_Sum.getInfo())

# presentLitter_Abs_Sum = presentLitter_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Dead wood and litter:', 'blue', attrs=['bold']),presentLitter_Abs_Sum.getInfo())

# potentialLitter_Abs_Sum = potentialLitter_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Dead wood and litter:', 'blue', attrs=['bold']),potentialLitter_Abs_Sum.getInfo())

# presentLitter_Lower_Sum = presentLitter_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Dead wood and litter Lower:', 'blue', attrs=['bold']),presentLitter_Lower_Sum.getInfo())

# presentLitter_Upper_Sum = presentLitter_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present Dead wood and litter Upper:', 'blue', attrs=['bold']),presentLitter_Upper_Sum.getInfo())

# potentialLitter_Lower_Sum = potentialLitter_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential Dead wood and litter Lower:', 'blue', attrs=['bold']),potentialLitter_Lower_Sum.getInfo())

# potentialLitter_Upper_Sum = potentialLitter_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)
# # print the estimation out
# print(colored('The potential Dead wood and litter Upper:', 'blue', attrs=['bold']),potentialLitter_Upper_Sum.getInfo())


In [11]:
# calculate the present PGB Lower and Upper
presentPGB_Lower_Abs = presentTGB_Lower_Abs.multiply(deadWoodLitterRatioLower)
presentPGB_Upper_Abs = presentTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper)
# calculate the potential PGB Lower and Upper
potentialPGB_Lower_Abs = potentialTGB_Lower_Abs.multiply(deadWoodLitterRatioLower)
potentialPGB_Upper_Abs = potentialTGB_Upper_Abs.multiply(deadWoodLitterRatioUpper)

presentPGB_D = presentAGB_Density.multiply(rootShootRatio.add(1)).multiply(deadWoodLitterRatio)
potentialPGB_D = potentialAGB_Density.multiply(rootShootRatio.add(1)).multiply(deadWoodLitterRatio)

# presentPGB_Lower_Sum = presentPGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present PGB Lower:', 'blue', attrs=['bold']),presentPGB_Lower_Sum.getInfo())

# presentPGB_Upper_Sum = presentPGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The present PGB Upper:', 'blue', attrs=['bold']),presentPGB_Upper_Sum.getInfo())

# potentialPGB_Lower_Sum = potentialPGB_Lower_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential PGB Lower:', 'blue', attrs=['bold']),potentialPGB_Lower_Sum.getInfo())

# potentialPGB_Upper_Sum = potentialPGB_Upper_Abs.multiply(biomeMask).reduceRegion(reducer = ee.Reducer.sum(),
#                                                        geometry = unboundedGeo,
#                                                        crs = 'EPSG:4326',
#                                                        crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
#                                                        maxPixels = 1e9)

# # print the estimation out
# print(colored('The potential PGB Upper:', 'blue', attrs=['bold']),potentialPGB_Upper_Sum.getInfo())



## 6 Export the upper and lower images to Assets

In [12]:
# load the carbon density layers
SandermannCarbonDiff = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_Diff_1km_Present_subtract_NoLU").unmask()
SandermannCarbonPresent = ee.Image("users/leonidmoore/ForestBiomass/SoilOrganicCarbonModel/SOCS_0_200cm_1km_Present").unmask()

# mask the diffrence layer
SandermannCarbonLoss = SandermannCarbonDiff.multiply(SandermannCarbonDiff.gt(0))

# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask = presentForestCover.gt(0)
potentialMask = potentialCoverAdjusted.gte(0.1)

# calculate the sum of the potential in soil with the consideration of forest cover
SandermannCarbonStockLoss = SandermannCarbonLoss.multiply(pixelArea).divide(1000000000).mask(biomeMask).mask(potentialMask).multiply(potentialCoverAdjusted)

# add the soil into the PGB as the total potential
potentialTotal_Abs = potentialPGB_Abs.add(SandermannCarbonStockLoss)
# compose those bands into an image
lowerUpperImage = presentAGB_Lower_Abs.rename('preAGB_Lower').addBands(presentAGB_Upper_Abs.rename('preAGB_Upper')).addBands(potentialAGB_Lower_Abs.rename('potAGB_Lower')).addBands(potentialAGB_Upper_Abs.rename('potAGB_Upper')).addBands(presentRoot_Lower_Abs.rename('preRoot_Lower')).addBands(presentRoot_Upper_Abs.rename('preRoot_Upper')).addBands(potentialRoot_Lower_Abs.rename('potRoot_Lower')).addBands(potentialRoot_Upper_Abs.rename('potRoot_Upper')).addBands(presentLitter_Lower_Abs.rename('preLitter_Lower')).addBands(presentLitter_Upper_Abs.rename('preLitter_Upper')).addBands(potentialLitter_Lower_Abs.rename('potLitter_Lower')).addBands(potentialLitter_Upper_Abs.rename('potLitter_Upper')).addBands(potentialTotal_Abs.rename('PotentialTotal'))

In [13]:
exportUpperLower = ee.batch.Export.image.toAsset(image = lowerUpperImage,
                                               description = 'WK2_Upper_Lower_Uncertainty_maps_Export',
                                               assetId = 'users/leonidmoore/ForestBiomass/UncertaintyFigure/WK2_Lower_Upper_Map',
                                               region = unboundedGeo,
                                               crs = 'EPSG:4326',
                                               crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                               maxPixels = 1e13)
# start the export task
exportUpperLower.start()
# show the task status
exportUpperLower.status()

{'state': 'READY',
 'description': 'WK2_Upper_Lower_Uncertainty_maps_Export',
 'creation_timestamp_ms': 1690807865941,
 'update_timestamp_ms': 1690807865941,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'G3C3PT4WDMMTNANBV4SZ42EC',
 'name': 'projects/earthengine-legacy/operations/G3C3PT4WDMMTNANBV4SZ42EC'}

## 7 Calculate the Abs for different parts at biome level

In [16]:
# Stack the absolute biomass layers into an Image.
absImage = presentAGB_Abs.rename('PresentAGB').addBands(potentialAGB_Abs.rename('PotentialAGB')).addBands(presentRoot_Abs.rename('PresentRoot')).addBands(potentialRoot_Abs.rename('PotentialRoot')).addBands(presentTGB_Abs.rename('PresentTGB')).addBands(potentialTGB_Abs.rename('PotentialTGB')).addBands(presentLitter_Abs.rename('PresentLitter')).addBands(potentialLitter_Abs.rename('PotentialLitter')).addBands(presentPGB_Abs.rename('PresentPGB')).addBands(potentialPGB_Abs.rename('PotentialPGB'))

# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    perBiomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absImage.mask(perBiomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['PresentAGB','PotentialAGB','PresentRoot','PotentialRoot','PresentTGB','PotentialTGB','PresentLitter','PotentialLitter','PresentPGB','PotentialPGB']).round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/WK2_Abs_for_diff_parts_at_Biome_Level.csv',header=True,mode='w+')
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,PresentAGB,PotentialAGB,PresentRoot,PotentialRoot,PresentTGB,PotentialTGB,PresentLitter,PotentialLitter,PresentPGB,PotentialPGB
0,180.7,227.5,48.0,59.6,228.7,287.2,50.3,63.2,279.0,350.4
1,7.9,18.3,2.0,4.7,9.9,23.0,2.2,5.1,12.0,28.1
2,2.8,5.2,0.7,1.3,3.5,6.6,0.8,1.4,4.2,8.0
3,39.2,61.4,10.0,15.5,49.2,77.0,16.2,25.4,65.5,102.4
4,19.3,23.4,5.1,6.2,24.4,29.6,8.0,9.8,32.5,39.4
5,35.7,44.7,9.6,12.0,45.3,56.7,36.2,45.3,81.5,102.1
6,44.3,93.7,13.5,28.4,57.9,122.1,12.7,26.9,70.6,148.9
7,3.7,22.5,1.1,6.6,4.8,29.1,1.6,9.6,6.4,38.7
8,2.2,3.8,0.7,1.1,2.8,5.0,0.6,1.1,3.4,6.1
9,3.4,10.1,1.0,2.7,4.4,12.8,1.4,4.2,5.8,17.0


In [35]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.

## 7 Calculate the Abs for different parts at biome level

In [16]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = presentAGB_Lower_Abs.rename('preAGB_Lower').addBands(presentAGB_Upper_Abs.rename('preAGB_Upper')).addBands(potentialAGB_Lower_Abs.rename('potAGB_Lower')).addBands(potentialAGB_Upper_Abs.rename('potAGB_Upper')).addBands(presentRoot_Lower_Abs.rename('preRoot_Lower')).addBands(presentRoot_Upper_Abs.rename('preRoot_Upper')).addBands(potentialRoot_Lower_Abs.rename('potRoot_Lower')).addBands(potentialRoot_Upper_Abs.rename('potRoot_Upper')).addBands(presentLitter_Lower_Abs.rename('preLitter_Lower')).addBands(presentLitter_Upper_Abs.rename('preLitter_Upper')).addBands(potentialLitter_Lower_Abs.rename('potLitter_Lower')).addBands(potentialLitter_Upper_Abs.rename('potLitter_Upper'))
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    perBiomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(perBiomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['preAGB_Lower','preAGB_Upper','potAGB_Lower','potAGB_Upper','preRoot_Lower','preRoot_Upper','potRoot_Lower','potRoot_Upper','preLitter_Lower','preLitter_Upper','potLitter_Lower','potLitter_Upper']).round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/WK2_Uncertainty_for_diff_parts_at_Biome_Level.csv',header=True,mode='w+')
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,preAGB_Lower,preAGB_Upper,potAGB_Lower,potAGB_Upper,preRoot_Lower,preRoot_Upper,potRoot_Lower,potRoot_Upper,preLitter_Lower,preLitter_Upper,potLitter_Lower,potLitter_Upper
0,179.4,181.9,204.7,235.6,37.6,59.1,42.7,74.9,32.6,72.3,37.1,93.2
1,7.8,7.9,15.6,20.9,1.6,2.4,3.2,6.2,1.4,3.1,2.8,8.1
2,2.7,2.8,4.6,5.9,0.6,0.8,1.0,1.7,0.5,1.1,0.8,2.3
3,38.8,39.7,51.8,64.7,8.8,11.4,11.7,18.6,14.3,18.9,19.0,30.8
4,19.0,19.5,17.1,22.7,4.4,5.9,4.0,6.9,7.0,9.4,6.3,10.9
5,35.4,36.0,37.9,46.4,8.7,10.7,9.4,13.9,30.0,43.9,32.1,56.7
6,43.9,44.8,78.1,109.2,11.4,15.7,20.0,38.5,8.3,18.2,14.7,44.3
7,3.7,3.8,18.7,26.0,0.9,1.2,4.8,8.9,1.4,1.9,7.0,12.9
8,2.1,2.2,3.2,4.4,0.6,0.8,0.8,1.5,0.4,0.9,0.6,1.8
9,3.4,3.5,8.0,12.0,0.8,1.1,1.8,3.7,1.2,1.7,2.9,5.8


In [ ]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.

## 8 Calculate the uncertainty for each land use types

In [17]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).multiply(oneSubtract.lt(0)).divide(sumCover.subtract(presentForestCover).multiply(oneSubtract.lt(0))).unmask();
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialForestCover.subtract(presentForestCover).mask(sumlandCover.neq(0)).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation)# the left positive pixels are real freeland pixels

In [18]:
# define the fucntion that can do the carbon partitionning into differen land use types
def absCalculationFunc(presentDensity,potentialDensity):
    # calculate the existing carbon, present potential carbon and absolute potential carbon in forests
    absoluteImage1 = presentDensity.multiply(pixelArea).multiply(presentMask).divide(1000000000).rename('Present')
    absoluteImage3 = potentialDensity.multiply(pixelArea).multiply(potentialMask).divide(1000000000).rename('AbsolutePotential')
    # get the sum of the potential covers
    potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

    trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
    ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialForestCover.divide(presentForestCover))
    #  get the real for conservation potential
    realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
    realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
    trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialForestCover))))

    conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialForestCover))
    conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialForestCover.subtract(presentForestCover)))

    absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

    trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

    absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
    absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
    absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
    absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
    absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')
    
    # Stack the absolute biomass layers into an Image.
    absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8)
    
    output = absPotentialImage.mask(biomeMask).reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    statisticTable = output.getInfo()
    # # transform into data frame
    outputTable = pd.DataFrame(statisticTable,index=['i',],columns =['Types','Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential']).round(1)
    # return the outputTable
    return outputTable

In [20]:
# for AGBs
resultAGB_Lower = absCalculationFunc(presentDensity = presentAGB_Lower,potentialDensity = potentialAGB_AdjLower)
resultAGB_Upper = absCalculationFunc(presentDensity = presentAGB_Upper,potentialDensity = potentialAGB_AdjUpper)

# For roots
presentRoot_Lower = presentAGB_Lower.multiply(rootShootRatioLower).mask(presentMask)
potentialRoot_Lower = potentialAGB_AdjLower.multiply(rootShootRatioLower).mask(potentialMask)
presentRoot_Upper = presentAGB_Upper.multiply(rootShootRatioUpper).mask(presentMask)
potentialRoot_Upper = potentialAGB_AdjUpper.multiply(rootShootRatioUpper).mask(potentialMask)

resultRoot_Lower = absCalculationFunc(presentDensity = presentRoot_Lower,potentialDensity = potentialRoot_Lower)
resultRoot_Upper = absCalculationFunc(presentDensity = presentRoot_Upper,potentialDensity = potentialRoot_Upper)
# for Litter
presentLitter_Lower = presentLitter_Lower_Abs.divide(pixelArea).multiply(1000000000).mask(presentMask)
potentialLitter_Lower = potentialLitter_Lower_Abs.divide(pixelArea).multiply(1000000000).mask(potentialMask)
presentLitter_Upper = presentLitter_Upper_Abs.divide(pixelArea).multiply(1000000000).mask(presentMask)
potentialLitter_Upper = potentialLitter_Upper_Abs.divide(pixelArea).multiply(1000000000).mask(potentialMask)

resultLitter_Lower = absCalculationFunc(presentDensity = presentLitter_Lower,potentialDensity = potentialLitter_Lower)
resultLitter_Upper = absCalculationFunc(presentDensity = presentLitter_Upper,potentialDensity = potentialLitter_Upper)
# for TGB
presentTGB_Lower = presentTGB_Lower_Abs.divide(pixelArea).multiply(1000000000).mask(presentMask)
potentialTGB_Lower = potentialTGB_Lower_Abs.divide(pixelArea).multiply(1000000000).mask(potentialMask)
presentTGB_Upper = presentTGB_Upper_Abs.divide(pixelArea).multiply(1000000000).mask(presentMask)
potentialTGB_Upper = potentialTGB_Upper_Abs.divide(pixelArea).multiply(1000000000).mask(potentialMask)

resultTGB_Lower = absCalculationFunc(presentDensity = presentTGB_Lower,potentialDensity = potentialTGB_Lower)
resultTGB_Upper = absCalculationFunc(presentDensity = presentTGB_Upper,potentialDensity = potentialTGB_Upper)
# For PGB
presentPGB_Lower = presentPGB_Lower_Abs.divide(pixelArea).multiply(1000000000).mask(presentMask)
potentialPGB_Lower = potentialPGB_Lower_Abs.divide(pixelArea).multiply(1000000000).mask(potentialMask)
presentPGB_Upper = presentPGB_Upper_Abs.divide(pixelArea).multiply(1000000000).mask(presentMask)
potentialPGB_Upper = potentialPGB_Upper_Abs.divide(pixelArea).multiply(1000000000).mask(potentialMask)

resultPGB_Lower = absCalculationFunc(presentDensity = presentPGB_Lower,potentialDensity = potentialPGB_Lower)
resultPGB_Upper = absCalculationFunc(presentDensity = presentPGB_Upper,potentialDensity = potentialPGB_Upper)




# concat all the tables into a data frame
concatTable = pd.concat([resultAGB_Lower,resultAGB_Upper,resultRoot_Lower,resultRoot_Upper,resultLitter_Lower,resultLitter_Upper,resultTGB_Lower,resultTGB_Upper,resultPGB_Lower,resultPGB_Upper])

finalTable = concatTable.assign(Types=['AGB_Lower',
                                       'AGB_Upper',
                                       'Root_Lower',
                                       'Root_Upper',
                                       'Litter_Lower',
                                       'Litter_Upper',
                                       'TGB_Lower',
                                       'TGB_Upper',
                                       'PGB_Lower',
                                       'PGB_Upper'])

In [21]:
finalTable.to_csv('BiomeLevelStatistics/StatisticsForModels/WK2_Landuse_type_Uncertainty.csv',header=True,mode='w+')
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
finalTable.head(10)

The biomass partition results in biome: 



,Types,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential
i,AGB_Lower,347.0,392.8,507.1,34.1,23.7,22.7,32.4,1.0
i,AGB_Upper,353.3,443.5,619.6,54.1,37.7,34.1,48.1,1.5
i,Root_Lower,78.0,88.4,114.7,4.1,2.5,4.5,5.4,0.1
i,Root_Upper,113.2,142.0,198.2,8.8,5.5,9.3,10.9,0.2
i,Litter_Lower,100.4,113.8,140.3,5.6,2.0,4.2,5.9,0.2
i,Litter_Upper,179.7,226.9,312.6,15.7,6.7,12.1,15.2,0.5
i,TGB_Lower,425.0,481.2,621.8,21.1,12.0,22.9,28.8,0.9
i,TGB_Upper,466.5,585.5,817.8,35.2,20.5,37.0,45.4,1.3
i,PGB_Lower,525.4,594.9,762.0,26.7,14.0,27.1,34.7,1.1
i,PGB_Upper,646.2,812.4,1130.4,50.9,27.2,49.0,60.6,1.8


In [ ]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.